# MAJOR CODE I AM WORKING ON ONLY VISUALIZATION LEFT

In [2]:
from PyQt5 import QtWidgets, QtGui, QtCore
import sys
import cv2
from PyQt5.QtCore import QTimer
from PyQt5.QtGui import QImage, QPixmap
import torch
from datetime import datetime
import time
import numpy as np
import mysql.connector
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QPushButton, QMessageBox, QLineEdit,QComboBox, QListWidget
import os
import pandas as pd
import requests

def get_geolocation(ip_address):
    url = f"http://ip-api.com/json/{ip_address}"
    response = requests.get(url)
    data = response.json()

    if data["status"] == "success":
        # Return latitude and longitude as floats
        return float(data['lat']), float(data['lon'])
    else:
        print("Could not retrieve geolocation information.")
        return None, None  # Return None values if there's an error



# Database connection
def connect_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="",  # Replace with your password
        database="viz_optilytics"  # Replace with your database name
    )
######################################################################################
# Load color reference CSV and create color lookup
index = ["color", "color_name", "hex", "R", "G", "B"]
csv = pd.read_csv('colors.csv', names=index, header=None)

def get_color_name(bgr):
    b, g, r = bgr
    minimum = 10000
    cname = "Unknown"
    for i in range(len(csv)):
        d = abs(r - int(csv.loc[i, "R"])) + abs(g - int(csv.loc[i, "G"])) + abs(b - int(csv.loc[i, "B"]))
        if d <= minimum:
            minimum = d
            cname = csv.loc[i, "color_name"]
    return cname

# Function to extract color from bounding box
def extract_color(frame, bbox):
    x1, y1, x2, y2 = bbox
    roi = frame[y1:y2, x1:x2]
    average_color = np.mean(roi, axis=(0, 1))
    color_name = get_color_name(average_color)
    return color_name
###############################################  

def save_vehicle_data(color, vehicle_type, timestamp, loc_lat, loc_lon):
    try:
        conn = connect_db()
        cursor = conn.cursor()
        print("I am savng the data to database")
        query = "INSERT INTO vehicles (color, vehicle_type, timestamp, loc_lat, loc_lon) VALUES (%s,%s,%s,%s,%s)"
        cursor.execute(query, (color, vehicle_type, timestamp, loc_lat,loc_lon))
        conn.commit()
        cursor.close()
        conn.close()

        print("The data has been saved successfully")
    except mysql.connector.Error as err:
        print(f"Error: {err}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        

def save_people_data(people_count, timestamp, loc_lat, loc_lon):
    try:
        conn = connect_db()
        cursor = conn.cursor()
        
        query = "INSERT INTO people (people_count, timestamp, loc_lat, loc_lon) VALUES (%s,%s,%s,%s)"
        cursor.execute(query, (people_count, timestamp, loc_lat,loc_lon))
        conn.commit()
        cursor.close()
        conn.close()

    except mysql.connector.Error as err:
        print(f"Error: {err}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        
# Save video to database with name
def save_video_to_db(video_path, video_name, start_time, duration, end_time):
    try:
        conn = connect_db()
        cursor = conn.cursor()

        # Read video file data
        with open(video_path, 'rb') as f:
            video_data = f.read()

        # Insert video data into database
        query = """
            INSERT INTO video_storage (video_name, video_file, start_time, duration, end_time)
            VALUES (%s, %s, %s, %s, %s)
        """
        cursor.execute(query, (video_name, video_data, start_time, duration, end_time))
        
        # Commit transaction
        conn.commit()
        # Clean up
        cursor.close()
        conn.close()

        # Optionally remove the file from the system
        os.remove(video_path)
        print("Video saved to database and deleted from the computer successfully.")

    except mysql.connector.Error as err:
        print(f"Database error: {err}")
    except FileNotFoundError:
        print("Error: The video file was not found.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        
# Search for videos based on date and time
def search_videos(search_term):
    try:
        conn = connect_db()
        cursor = conn.cursor()

        # SQL query to search for videos by date or specific datetime
        query = "SELECT video_name FROM video_storage WHERE video_name LIKE %s"
        cursor.execute(query, (f"{search_term}%",))  # '%' allows for matching any suffix

        results = cursor.fetchall()
        cursor.close()
        conn.close()

        return [result[0] for result in results]  # Extract video names from results

    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return []

class VizOptilyticsApp(QtWidgets.QWidget):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Viz Optilytics")
        self.setGeometry(0, 0, 1200, 700)

        # Initialize QTimer (fixing the timer error)
        self.timer = QTimer(self)  # Initialize the QTimer object

        # Initialize video capture object
        self.video_capture = None

        # Detection states
        self.detecting_person = False
        self.detecting_vehicle = False

         # Track detected persons and duration
        self.person_detected_time = None  # To track when a person is first detected
        self.alert_generated = False  # To track if alert is already generated
        
        # Load YOLOv5 model
        self.model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # Load YOLOv5 model

         # Initialize alert labels for dynamic updates
        self.alert_label = None
        self.duration_label = None
        
        self.detection_grace_period = 10  # Number of frames to maintain detection box
        self.frames_since_last_detection = 0
        self.last_detection_box = None  # Store the last detected bounding box

        
        self.vehicle_positions = {}  # Stores vehicle ID -> (last position, detection start time)
        self.stationary_alert_generated = {}  # To track if stationary alert has been generated

        self.video1 = False
        self.video2 = False
        self.video3 = False
        self.real_time_stream = False
        self.record_flag = False

        self.person_count = 0  # Track number of people detected
        
        self.initUI()
    def initUI(self):
        # Main Layout
        main_layout = QtWidgets.QVBoxLayout(self)
        main_layout.setContentsMargins(50, 50, 50, 30)

        # Top Section: Title and Menu
        top_bar = QtWidgets.QHBoxLayout()
#         top_bar.addStretch(1)

        title = QtWidgets.QLabel("Viz Optilytics")
        title.setFont(QtGui.QFont("Arial", 28, QtGui.QFont.Bold))
        title.setStyleSheet("color: #FFD700;")
        title.setAlignment(QtCore.Qt.AlignLeft)

        top_bar.addWidget(title)
        top_bar.addStretch(2)

        # Create an editable QComboBox for search
        self.search_combo = QComboBox()
        self.search_combo.setEditable(True)  # Make the combo box editable
        self.search_combo.setPlaceholderText("Enter date (YYYY-MM-DD) or datetime (YYYY-MM-DD_HH-MM-SS)")
        top_bar.addWidget(self.search_combo)

        top_bar.addStretch(1)
        
        menu_button = QtWidgets.QPushButton("☰")
        menu_button.setStyleSheet("background-color: #333333; color: #FFD700; font-size: 20px; border-radius: 10px;")
        menu_button.setFixedSize(50, 50)
        menu_button.setMenu(self.create_menu())

        top_bar.addWidget(menu_button)
        main_layout.addLayout(top_bar)

        title_spacer = QtWidgets.QSpacerItem(20, 40, QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Fixed)
        main_layout.addItem(title_spacer)

        # Horizontal layout split: Videos, Buttons, Alerts, etc.
        content_layout = QtWidgets.QHBoxLayout()
        content_layout.setSpacing(20)

        # Video Buttons and Info Section
        main_vertical_layout = QtWidgets.QVBoxLayout()

        video_list_layout = QtWidgets.QVBoxLayout()
        recorded_videos_label = QtWidgets.QLabel("Recorded Videos")
        recorded_videos_label.setFont(QtGui.QFont("Arial", 16, QtGui.QFont.Bold))
        recorded_videos_label.setStyleSheet("color: white;")
        video_list_layout.addWidget(recorded_videos_label)

        # Video buttons
        for i in range(3):
            video_btn = QtWidgets.QPushButton(f"Video {i + 1}")
            video_btn.setStyleSheet("background-color: #555555; color: white; border-radius: 8px; padding: 8px;")
            video_btn.setFixedWidth(200)
            video_btn.setSizePolicy(QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Fixed)

            # Connect each button to a separate function based on the index
            if i == 0:
                video_btn.clicked.connect(self.play_video_1)
            elif i == 1:
                video_btn.clicked.connect(self.play_video_2)
            elif i == 2:
                video_btn.clicked.connect(self.play_video_3)

            video_list_layout.addWidget(video_btn)

        main_vertical_layout.addLayout(video_list_layout)

        detection_info_widget = QtWidgets.QWidget()
        detection_info_layout = QtWidgets.QVBoxLayout(detection_info_widget)
        detection_info_layout.setSpacing(0)
        detection_info_widget.setStyleSheet("background-color: #444444; border-radius: 10px;")

        date_time_label = QtWidgets.QLabel("Date: 2024-10-11\nTime: 12:45 PM")
        date_time_label.setStyleSheet("color: white; font-size: 14px; padding: 8px;")
        detection_info_layout.addWidget(date_time_label)

        detection_status = QtWidgets.QLabel("Status: Vehicle Detected")
        detection_status.setStyleSheet("color: white; font-size: 14px; padding: 8px;")
        detection_info_layout.addWidget(detection_status)
        main_vertical_layout.addWidget(detection_info_widget)

        # Real-time streaming button
        real_time_btn = QtWidgets.QPushButton("Real-Time Streaming")
        real_time_btn.setStyleSheet("background-color: #666666; padding: 10px; font-size: 14px; border-radius: 8px;")
        real_time_btn.setFixedWidth(200)
        real_time_btn.setSizePolicy(QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Fixed)
        real_time_btn.clicked.connect(self.start_real_time_stream)
        main_vertical_layout.addWidget(real_time_btn)

        # Analytics button
        analytics_btn = QtWidgets.QPushButton("Analytics")
        analytics_btn.setStyleSheet("background-color: #666666; padding: 10px; font-size: 14px; border-radius: 8px;")
        analytics_btn.setFixedWidth(200)
        analytics_btn.setSizePolicy(QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Fixed)
        analytics_btn.clicked.connect(self.show_analytics)
        main_vertical_layout.addWidget(analytics_btn)

        content_layout.addLayout(main_vertical_layout)

        # Video display area and other buttons
        main_left_layout = QtWidgets.QVBoxLayout()

        video_layout = QtWidgets.QVBoxLayout()
        video_controls_layout = QtWidgets.QHBoxLayout()

        # Create a layout to position the close button at the top-right corner
        close_button = QtWidgets.QPushButton("X")
        close_button.setStyleSheet("background-color: red; color: white; font-size: 18px; border-radius: 10px;")
        close_button.setFixedSize(30, 30)
        close_button.clicked.connect(self.stop_stream)

        # Create a wrapper for the video feed and the button
        video_feed_wrapper = QtWidgets.QWidget()
        video_feed_layout = QtWidgets.QVBoxLayout(video_feed_wrapper)
        video_feed_layout.setContentsMargins(0, 0, 0, 0)  # Remove margins for precise alignment

        # Create a layout for the close button at the top-right corner
        close_button_layout = QtWidgets.QHBoxLayout()
        close_button_layout.addStretch(1)  # Add stretch to push the button to the right
        close_button_layout.addWidget(close_button)

        video_feed_layout.addLayout(close_button_layout)  # Add the close button layout
        self.video_feed = QtWidgets.QLabel("Video Stream")
        self.video_feed.setAlignment(QtCore.Qt.AlignCenter)
        self.video_feed.setStyleSheet("background-color: white; color: black; border: 2px solid;")
        self.video_feed.setFixedSize(970, 480)  # Set a fixed size for the video feed
        self.video_feed.setScaledContents(True)
        self.video_feed.setSizePolicy(QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Expanding)

        video_feed_layout.addWidget(self.video_feed)  # Add the video feed below the button

        # Add the video feed wrapper to the main video layout
        video_layout.addWidget(video_feed_wrapper)

        # Detection buttons
        detection_buttons_layout = QtWidgets.QHBoxLayout()
        self.record_btn = QtWidgets.QPushButton("Start Record")
        self.record_btn.setStyleSheet("background-color: #666666; color: white; padding: 10px; font-size: 14px; border-radius: 8px;")
        self.record_btn.setSizePolicy(QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Fixed)
        self.record_btn.clicked.connect(self.toggle_record)
        detection_buttons_layout.addWidget(self.record_btn)

       # Assigning person_detection_btn as an instance variable
        self.person_detection_btn = QtWidgets.QPushButton("Person Detection")
        self.person_detection_btn.setStyleSheet("background-color: #666666; color: white; padding: 10px; font-size: 14px; border-radius: 8px;")
        self.person_detection_btn.setSizePolicy(QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Fixed)
        self.person_detection_btn.clicked.connect(self.toggle_person_detection)
        detection_buttons_layout.addWidget(self.person_detection_btn)

        self.vehicle_detection_btn = QtWidgets.QPushButton("Vehicle Detection")
        self.vehicle_detection_btn.setStyleSheet("background-color: #666666; color: white; padding: 10px; font-size: 14px; border-radius: 8px;")
        self.vehicle_detection_btn.setSizePolicy(QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Fixed)
        self.vehicle_detection_btn.clicked.connect(self.toggle_vehicle_detection)
        detection_buttons_layout.addWidget(self.vehicle_detection_btn)

        video_layout.addLayout(detection_buttons_layout)
        main_left_layout.addLayout(video_layout)

        # Alerts section
        alerts_layout = QtWidgets.QHBoxLayout()
        self.alert_label = QtWidgets.QLabel("Alert: No Detection")
        self.alert_label.setFixedSize(450, 37)  # Set fixed size for the alert label
        self.alert_label.setAlignment(QtCore.Qt.AlignCenter)
        self.alert_label.setStyleSheet("background-color: green; color: white; padding: 10px; font-size: 14px; border-radius: 8px;")
        alerts_layout.addWidget(self.alert_label)

        self.duration_label = QtWidgets.QLabel("Time Duration: 00:00:00")
        self.duration_label.setAlignment(QtCore.Qt.AlignCenter)
        self.duration_label.setFixedSize(450, 37)  # Set fixed size for the alert label
        self.duration_label.setStyleSheet("background-color: green; color: white; padding: 10px; font-size: 14px; border-radius: 8px;")
        alerts_layout.addWidget(self.duration_label)

        main_left_layout.addLayout(alerts_layout)
        content_layout.addLayout(main_left_layout)

        main_layout.addLayout(content_layout)
        self.setStyleSheet("background-color: #222222; color: white;")

        self.setLayout(main_layout)
        # Connect the textChanged signal to the search function with a timer
        self.search_combo.lineEdit().textChanged.connect(self.on_text_changed)
        self.search_timer = QTimer()
        self.search_timer.setInterval(300)  # Delay in milliseconds (300ms)
        self.search_timer.timeout.connect(self.perform_search)
        self.search_term = ""
        
    def on_text_changed(self):
        self.search_term = self.search_combo.lineEdit().text()
        self.search_timer.start()  # Restart the timer on every keystroke

    def perform_search(self):
        self.search_timer.stop()  # Stop the timer
        if not self.search_term:
            self.search_combo.clear()  # Clear results if input is empty
            return

        results = search_videos(self.search_term)
        self.search_combo.clear()  # Clear previous results
        if results:
            self.search_combo.addItems(results)  # Add results to the dropdown
            self.search_combo.showPopup()  # Show the dropdown with results
        else:
            self.search_combo.addItem("No videos found matching the search criteria.")

    
    def toggle_record(self):
        if not self.record_flag:
            self.record_flag = True
            self.record_btn.setText("Stop Record")
            self.record_options()
        else:
            self.record_flag = False
            self.record_btn.setText("Start Record")

    def record_options(self):
        msg_box = QMessageBox()
        msg_box.setWindowTitle("Choose Recording Type")
        msg_box.setText("Do you want to record the full video or use optimal storage?")
        full_button = msg_box.addButton("Full Video", QMessageBox.YesRole)
        optimal_button = msg_box.addButton("Optimal Storage", QMessageBox.NoRole)
        msg_box.exec_()
        
        if msg_box.clickedButton() == full_button:
            self.record_and_save_video(optimal=False)
        elif msg_box.clickedButton() == optimal_button:
            self.record_and_save_video(optimal=True)

    def record_and_save_video(self, optimal):
        now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        output_file = f"{now}.avi"
        self.video_display(output_file, optimal)  # Start the video display and recording

        # Notify the user that the recording has started
        QMessageBox.information(self, "Recording Started", "Video recording in progress...")
        # Store the start time
        self.start_time = datetime.now()
        # Connect the timer to check recording status
        self.timer.timeout.connect(lambda: self.check_recording_status(output_file, now, self.start_time))

    def check_recording_status(self, output_file, video_name, start_time):
        if not self.record_flag:  # Check if recording is complete
            self.timer.stop()  # Stop the timer
            # Calculate the end time and duration
            end_time = datetime.now()
            duration = end_time - start_time  # Calculate duration as a timedelta
            # Release the video writer if it's active
            if hasattr(self, 'out') and self.out is not None:
                self.out.release()
            # Format times for display or database storage
            start_time_str = start_time.strftime("%H:%M:%S")
            end_time_str = end_time.strftime("%H:%M:%S")
            duration_seconds = int(duration.total_seconds())
            duration_hours, remainder = divmod(duration_seconds, 3600)
            duration_minutes, duration_seconds = divmod(remainder, 60)
            duration_str = f"{duration_hours:02}:{duration_minutes:02}:{duration_seconds:02}"

            # Save the video to the database with details
            save_video_to_db(output_file, video_name, start_time_str, duration_str, end_time_str)
            print(f"the vdeo name is : {video_name}")
            # Notify the user that recording is complete
            QMessageBox.information(self, "Recording Complete", "Video recorded and saved to database successfully.")
            self.video_display()
            
    def video_display(self, output_path=None, optimal=False):
#         self.optimal = optimal
        if self.real_time_stream:
            self.video_capture = cv2.VideoCapture(0)
        elif self.video1:
            self.video_capture = cv2.VideoCapture(r"C:\Users\mahnoor\Downloads\853889-hd_1920_1080_25fps.mp4")
        elif self.video2:
            self.video_capture = cv2.VideoCapture(r"C:\Users\mahnoor\Downloads\2273134-hd_1280_720_30fps.mp4")
        elif self.video3:
            self.video_capture = cv2.VideoCapture(r"C:\Users\mahnoor\Downloads\12494476_3840_2160_50fps.mp4")
        if self.record_flag:
            fourcc = cv2.VideoWriter_fourcc(*'XVID')
            self.out = cv2.VideoWriter(output_path, fourcc, 20.0, (640, 480))
        ret, prev_frame = self.video_capture.read()
        if not ret:
            print("Error: Unable to access the camera")
            return
        if self.record_flag:
            self.prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY) if optimal else None
        # Disconnect previous connections to avoid duplicates
        try:
            self.timer.timeout.disconnect()
        except TypeError:
            # No existing connection to disconnect
            pass
        # Connect the update_frame method properly without calling it immediately
        self.timer.timeout.connect(lambda: self.update_frame(optimal))
        self.timer.start(20)
    
    def update_frame(self, optimal=False):
        ret, frame = self.video_capture.read()
        if ret:
            if self.record_flag and hasattr(self, 'out') and self.out is not None:
                if optimal:
                    # Convert current frame to grayscale for comparison
                    curr_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                    # Compute the absolute difference between current and previous frames
                    diff = cv2.absdiff(self.prev_gray, curr_gray)
                    _, thresh = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)
                    non_zero_count = cv2.countNonZero(thresh)

                    # Save the frame only if there is significant change
                    if non_zero_count > 10000:
                        self.out.write(frame)
                        self.prev_gray = curr_gray  # Update the previous frame
                else:
                    # Full video recording, save every frame
                    self.out.write(frame)

            # Convert the frame to RGB for display
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            if self.detecting_person:
                frame = self.detect_person(frame)
            if self.detecting_vehicle:
                frame = self.detect_vehicle(frame)

            # Get the frame dimensions and convert to QImage
            h, w, ch = frame.shape
            bytes_per_line = ch * w
            qimg = QImage(frame.data, w, h, bytes_per_line, QImage.Format_RGB888)

            self.video_feed.setPixmap(QPixmap.fromImage(qimg))
        else:
            print("Error: Unable to read video frame")

    def stop_stream(self):
        print("Stopping video stream")
        if self.video_capture is not None:
            self.video_capture.release()
#             self.out.release()
            self.video_capture = None
            self.video_feed.clear()
            self.timer.stop()

    def toggle_person_detection(self):
        """Toggle person detection on or off."""
        self.detecting_person = not self.detecting_person  # Toggle detection state

        if self.detecting_person:
            self.person_detection_btn.setText("Stop Person Detection")
        else:
            self.person_detection_btn.setText("Start Person Detection")

            # Reset detection states when stopping
            self.person_detected_time = None
            self.alert_generated = False
            self.alert_label.setText("Alert: No Detection")
            self.alert_label.setStyleSheet("background-color: green; color: white;")
            self.duration_label.setText("Time Duration: 00:00:00")
            self.duration_label.setStyleSheet("background-color: green; color: white;")


    def detect_person(self, frame):
        # Perform inference
        results = self.model(frame)
        predictions = results.pred[0]

        person_count = 0
        for *box, conf, cls in predictions:
            if int(cls) == 0:  # Class index for person
                person_count += 1
                x1, y1, x2, y2 = map(int, box)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw bounding box
                cv2.putText(frame, f'Person {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Update the number of people detected in the frame
        self.person_count = person_count

        # Check if person count exceeds 2
        if person_count > 2:
            self.update_person_detection_time()
        else:
            self.reset_person_detection()

        return frame

    def update_person_detection_time(self):
        """Track how long multiple people have been in the frame and update alert if needed."""
        if self.person_detected_time is None:
            self.person_detected_time = time.time()  # Start the timer

        duration = time.time() - self.person_detected_time

        # Format the duration
        hours, rem = divmod(duration, 3600)
        minutes, seconds = divmod(rem, 60)
        duration_str = "{:02}:{:02}:{:02}".format(int(hours), int(minutes), int(seconds))

        # Update duration label
        self.duration_label.setText(f"Time Duration: {duration_str}")

        # Check if detection duration exceeds the threshold (5 minutes or 300 seconds)
        if duration > 5 and not self.alert_generated:
            self.alert_label.setText("ALERT: More than 2 people detected for over 5 seconds!")
            self.alert_label.setStyleSheet("background-color: red; color: white;")
            self.duration_label.setStyleSheet("background-color: red; color: white;")
            self.alert_generated = True
            # Print head count and formatted time
            formatted_time = datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
            print(f"The number of head count is {self.person_count}")
            print(f"The time of detection is {formatted_time}")
            loc_lat, loc_lon = get_geolocation("182.183.56.95")
            save_people_data(self.person_count, formatted_time, loc_lat, loc_lon)

    def reset_person_detection(self):
        """Reset the detection state if fewer than three people are found."""
        if self.person_detected_time is not None:
            self.person_detected_time = None  # Reset the timer
            self.duration_label.setText("Time Duration: 00:00:00")
            self.alert_label.setText("Alert: No Detection")
            self.alert_label.setStyleSheet("background-color: green; color: white;")
            self.duration_label.setStyleSheet("background-color: green; color: white;")
            self.alert_generated = False
            self.person_count = 0  # Reset the person count to zero


            
    def detect_vehicle(self, frame):
        # Perform inference using YOLOv5 model
        results = self.model(frame)

        # Get predictions
        predictions = results.pred[0]

        current_time = time.time()  # Get the current time
        vehicle_detected = False  # Flag to check if any vehicle is detected

        # Dictionary to map class ids to vehicle types
        vehicle_class_map = {
            2: "Car",        # Car
            3: "Motorcycle", # Motorcycle
            5: "Bus",       # Bus
            7: "Truck"      # Truck
        }

        for *box, conf, cls in predictions:
            cls = int(cls)  # Convert cls to an integer for checking
            box = list(map(int, box))  # Ensure box contains integers
#             print(f"Bounding Box: {box}, Type: {type(box)}")  # Debugging statement

            if cls in vehicle_class_map:  # Only process valid vehicle classes
                vehicle_type = vehicle_class_map[cls]
                x1, y1, x2, y2 = box  # Unpacking the corrected box
                
                # Calculate the center of the bounding box
                vehicle_center = ((x1 + x2) // 2, (y1 + y2) // 2)
                vehicle_detected = True  # Set the flag to True since we detected a vehicle

                # Use the center position to track vehicle movement
                if vehicle_type not in self.vehicle_positions:
                    # If this is a new vehicle, start tracking it
                    self.vehicle_positions[vehicle_type] = (vehicle_center, current_time)
                    self.stationary_alert_generated[vehicle_type] = False
                else:
                    # Get the previous position and detection start time
                    prev_position, start_time = self.vehicle_positions[vehicle_type]

                    # Calculate movement distance between current and previous position
                    movement_distance = np.linalg.norm(np.array(vehicle_center) - np.array(prev_position))

                    # Define a movement threshold (e.g., 20 pixels)
                    movement_threshold = 20

                    if movement_distance < movement_threshold:
                        # If the vehicle has not moved significantly, check how long it's been stationary
                        stationary_duration = current_time - start_time

                        # Update the time duration label
                        self.duration_label.setText(f"Vehicle {vehicle_type} stationary for {int(stationary_duration)} seconds")

                        # Check if the stationary duration exceeds 5 minutes (300 seconds)
                        if stationary_duration > 5 and not self.stationary_alert_generated[vehicle_type]:
                            self.alert_label.setText(f"ALERT: {vehicle_type} stationary for over 5 minutes!")
                            self.alert_label.setStyleSheet("background-color: red; color: white;")
                            self.duration_label.setStyleSheet("background-color: red; color: white;")
                            self.stationary_alert_generated[vehicle_type] = True  # Mark that the alert has been generated
                            color = extract_color(frame, box)  # Corrected bounding box variable
                            timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
                            loc_lat, loc_lon = get_geolocation("182.183.56.95")  # Consider making IP dynamic
                            print("Saving the data in database")
                            print(f"color: {color}/n type: {vehicle_type} /n timestamp : {timestamp} lat_lon : {loc_lat} _ {loc_lon}")
                            save_vehicle_data(color, vehicle_type, timestamp, loc_lat, loc_lon)
                    else:
                        # If the vehicle moved, reset the detection start time
                        self.vehicle_positions[vehicle_type] = (vehicle_center, current_time)
                        self.stationary_alert_generated[vehicle_type] = False

                # Draw the bounding box and label the vehicle
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f'{vehicle_type} {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # If no vehicle is detected, reset the alert and duration labels
        if not vehicle_detected:
            self.alert_label.setText("No vehicle detected")
            self.alert_label.setStyleSheet("background-color: green; color: white;")
            self.duration_label.setText("No detection duration: 00 seconds")
            self.duration_label.setStyleSheet("background-color: green; color: white;")
        return frame
    
    
    def toggle_vehicle_detection(self):
        """Toggle vehicle detection on and off."""
        self.detecting_vehicle = not self.detecting_vehicle  # Toggle the state
        if self.detecting_vehicle:
            self.vehicle_detection_btn.setText("Stop vehicle Detection")
        else:
            self.vehicle_detection_btn.setText("Start vehicle Detection")
            self.alert_generated = False
            self.alert_label.setText("Alert: No Detection")
            self.alert_label.setStyleSheet("background-color: green; color: white;")
            self.duration_label.setText("Time Duration: 00:00:00")
            self.duration_label.setStyleSheet("background-color: green; color: white;")

    def create_menu(self):
        menu = QtWidgets.QMenu(self)
        menu.addAction("Option 1", self.option1_action)
        menu.addAction("Option 2", self.option2_action)
        return menu

    def start_real_time_stream(self):
        print("Starting real-time video stream")
        self.real_time_stream=True
        # Start video capture (assuming you have a webcam at index 0)
        self.video_display()  # Call video_feed to initiate video capture
            
   # Define the three separate play video functions
    def play_video_1(self):
        print("Playing Video 1")
        self.real_time_stream=False
        self.video1 = True  # Set instance variable
        self.video2 = False  # Reset other video flags
        self.video3 = False
        self.stop_stream()  # Stop any current stream before starting a video
        self.video_display()  # Call video_feed to start playing the video

    def play_video_2(self):
        print("Playing Video 2")
        self.real_time_stream=False
        self.video1 = False  # Reset other video flags
        self.video2 = True
        self.video3 = False
        self.stop_stream()  # Stop any current stream before starting a video
        self.video_display()  # Call video_feed to start playing the video

    def play_video_3(self):
        print("Playing Video 3")
        self.real_time_stream=False
        self.video1 = False  # Reset other video flags
        self.video2 = False
        self.video3 = True
        self.stop_stream()  # Stop any current stream before starting a video
        self.video_display()  # Call video_feed to start playing the video

    # Add your logic to play video 3 here
    def show_analytics(self):
        print("Showing analytics")

    def option1_action(self):
        print("Option 1 selected")

    def option2_action(self):
        print("Option 2 selected")


if __name__ == '__main__':
    app = QtWidgets.QApplication(sys.argv)
    window = VizOptilyticsApp()
    window.show()
    sys.exit(app.exec_())


Using cache found in C:\Users\mahnoor/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-3-25 Python-3.11.4 torch-2.2.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Playing Video 1
Stopping video stream
The number of head count is 39
The time of detection is 2024-11-04 12:48:26
Playing Video 3
Stopping video stream
Bounding Box: [3, 951, 1010, 1746], Type: <class 'list'>
Bounding Box: [1222, 1142, 2310, 1677], Type: <class 'list'>
Bounding Box: [1903, 1207, 2173, 1697], Type: <class 'list'>
Bounding Box: [1, 984, 1011, 1754], Type: <class 'list'>
Bounding Box: [2954, 1332, 3131, 1564], Type: <class 'list'>
Bounding Box: [2092, 1191, 2235, 1317], Type: <class 'list'>
Bounding Box: [2742, 1069, 3146, 1563], Type: <class 'list'>
Bounding Box: [1244, 1134, 2294, 1671], Type: <class 'list'>
Bounding Box: [1925, 1204, 2218, 1698], Type: <class 'list'>
Bounding Box: [3, 950, 1012, 1746], Type: <class 'list'>
Bounding Box: [0, 985, 1011, 1753], Type: <class 'list'>
Bounding Box: [2955, 1332, 3129, 1563], Type: <class 'list'>
Bounding Box: [2657, 1027, 2979, 1564], Type: <class 'list'>
Bounding Box: [2092, 1194, 2232, 1311], Type: <class 'list'>
Bounding B

Bounding Box: [1544, 1145, 2812, 1657], Type: <class 'list'>
Bounding Box: [2376, 1210, 2756, 1608], Type: <class 'list'>
Bounding Box: [1, 979, 1011, 1753], Type: <class 'list'>
Bounding Box: [2, 945, 1015, 1747], Type: <class 'list'>
Bounding Box: [1412, 1175, 1507, 1369], Type: <class 'list'>
Bounding Box: [2372, 1217, 2879, 1605], Type: <class 'list'>
Bounding Box: [2961, 1371, 3123, 1565], Type: <class 'list'>
Bounding Box: [1500, 1504, 3737, 2160], Type: <class 'list'>
Stopping video stream
Starting real-time video stream
Video saved to database and deleted from the computer successfully.
the vdeo name is : 2024-11-04_12-49-12
Stopping video stream


SystemExit: 0

# CHECKING THE VIDEOS RECORDED BY RETRIVING THEM

In [1]:
import sys
import cv2
import mysql.connector
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QPushButton, QLabel, QFileDialog, QMessageBox, QLineEdit
from PyQt5.QtMultimedia import QMediaPlayer, QMediaContent
from PyQt5.QtCore import QUrl
from PyQt5.QtMultimediaWidgets import QVideoWidget
import os

# Database connection
def connect_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="",  # Replace with your password
        database="viz_optilytics"  # Replace with your database name
    )


# Retrieve video from database using video name
def retrieve_video_from_db(video_name):
    try:
        conn = connect_db()
        cursor = conn.cursor()

        query = "SELECT video_file FROM video_storage WHERE video_name = %s"
        cursor.execute(query, (video_name,))
        result = cursor.fetchone()

        if result:
            video_data = result[0]
            cursor.close()
            conn.close()
            print("Video retrieved successfully.")
            return video_data
        else:
            print("No video found with that name.")
            return None
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None



# PyQt5 UI
class VideoApp(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('Video Recorder and Player')
        self.setGeometry(100, 100, 800, 600)

        self.layout = QVBoxLayout()

        

        self.retrieve_btn = QPushButton('Retrieve and Play Video')
        self.retrieve_btn.clicked.connect(self.retrieve_and_play_video)

        self.stop_btn = QPushButton('Stop Video')  # Stop video button
        self.stop_btn.clicked.connect(self.stop_video)

        self.player = QMediaPlayer(None, QMediaPlayer.VideoSurface)
        self.video_widget = QVideoWidget()

        # Text input for video name
        self.video_name_input = QLineEdit()
        self.video_name_input.setPlaceholderText("Enter video name")


        self.layout.addWidget(self.video_name_input)  # Add the input field to the layout
        self.layout.addWidget(self.retrieve_btn)
        self.layout.addWidget(self.stop_btn)  # Add stop button to layout
        self.layout.addWidget(self.video_widget)

        self.setLayout(self.layout)
        self.player.setVideoOutput(self.video_widget)

    def retrieve_and_play_video(self):
        # Get the video name from the input field
        video_name = self.video_name_input.text()

        if video_name:
            video_data = retrieve_video_from_db(video_name)  # Get the video data
            if video_data:
                # Save the video data temporarily to play it
                output_path = "temp_video.avi"  # Use a temporary file name
                with open(output_path, 'wb') as f:
                    f.write(video_data)
                self.play_video(output_path)  # Play the video from the temporary file
            else:
                QMessageBox.warning(self, "Error", "No video found.")
        else:
            QMessageBox.warning(self, "Invalid Input", "Please enter a valid video name.")

    def play_video(self, video_path):
        self.player.setMedia(QMediaContent(QUrl.fromLocalFile(video_path)))
        self.player.play()

    def stop_video(self):
        self.player.stop()
        self.player.setMedia(QMediaContent())  # Clear the media

# Main execution
if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = VideoApp()
    window.show()
    sys.exit(app.exec_())


Video retrieved successfully.


SystemExit: 0

C:\Users\mahnoor\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
